In [7]:
import asyncio
import urllib.request, json
import websockets
import pandas as pd
import nest_asyncio

In [8]:
nest_asyncio.apply()

In [9]:
def make_dataframe(url, param, index_name):
    #takes url for given deribit method we want to get and makes into a data frame
    #we do this in the notebook since jupyter takes up a thread for asyncio and 
    #I don't know how to fix that by using multithreaded stuff yet
    with urllib.request.urlopen(url) as url:
        data = json.loads(url.read().decode())
    data = pd.DataFrame(data[param]).set_index(index_name)
    return data

In [10]:
make_dataframe("https://test.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&kind=future", 'result', 'instrument_name')

,volume_usd,volume,quote_currency,price_change,open_interest,mid_price,mark_price,low,last,high,estimated_delivery_price,creation_timestamp,bid_price,base_currency,ask_price,funding_8h,current_funding
instrument_name,,,,,,,,,,,,,,,,,
BTC-25JUN21,1.402436e+07,367.83,USD,5.8344,69194660,36205.75,36235.11,33653.5,37032.5,41302.0,34172.89,1611961926556,36177.0,BTC,36234.5,NaN,NaN
BTC-26MAR21,7.252970e+07,1976.41,USD,3.1940,264198130,35192.25,35179.99,32621.0,35135.5,40106.5,34172.89,1611961926556,35184.5,BTC,35200.0,NaN,NaN
BTC-24SEP21,1.284052e+07,337.80,USD,6.8390,240133410,37293.50,37364.53,34448.0,38078.5,41995.5,34172.89,1611961926556,37230.0,BTC,37357.0,NaN,NaN
BTC-PERPETUAL,1.260354e+09,34809.23,USD,2.0395,1850390972,34195.25,34194.06,32000.0,34197.0,39123.0,34172.89,1611961926556,34193.5,BTC,34197.0,0.000494,0.00012


In [18]:
msg = \
    {"jsonrpc": "2.0",
     "method": "public/subscribe",
     "id": 42,
     "params": {
        "channels": ["markprice.options.btc_usd"]}
    }

async def call_api(msg):
    async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           data = response
           data = json.loads(data)
           #used_list = [i for i in data['params']]
           #used_list = pd.DataFrame.from_records(used_list)
           #data.keys()
           from pandas.io.json import json_normalize
           #data_normalized = json_normalize(data)
           if('result' in data):
               pass
           else:
               data_norm = json_normalize(data['params']['data'])
               print(data_norm)
               #await websocket.close()
               await websocket.close()
               return data_norm 
                
                
           #data = pd.io.json.json_normalize(response, record_path = [['params', 'data']])
           #data = pd.json_normalize(data)
           #data = pd.DataFrame(data['result']).set_index('instrument_name')
           print("\n\n")
           # do something with the notifications...

In [19]:
data = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(msg)))




     mark_price        iv      instrument_name  synthetic_future.mark_price  \
0      0.314776  0.883069  BTC-24SEP21-40000-P                          NaN   
1      0.000720  1.263131  BTC-26MAR21-11000-P                          NaN   
2      0.200256  0.856168  BTC-25JUN21-36000-P                          NaN   
3      0.033488  0.886022  BTC-26FEB21-30000-P                 35284.045028   
4      0.007926  1.258979   BTC-24SEP21-7000-P                          NaN   
..          ...       ...                  ...                          ...   
215    0.897701  1.438009   BTC-24SEP21-4000-C                          NaN   
216    0.143316  0.859497  BTC-26FEB21-32000-C                 35284.045028   
217    0.000003  0.843868  BTC-30JAN21-32000-P                 34760.751306   
218    0.749695  1.126150  BTC-24SEP21-10000-C                          NaN   
219    0.013698  1.126150  BTC-24SEP21-10000-P                          NaN   

    synthetic_future.instrument_name  
0        

/home/tjang/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [24]:
data.describe()

,mark_price,iv,synthetic_future.mark_price
count,2.200000e+02,220.000000,102.000000
mean,2.630576e-01,0.995255,35418.967854
std,4.043263e-01,0.252631,1059.600470
min,1.299362e-07,0.599155,34760.751306
25%,1.339710e-02,0.871313,34780.132554
50%,1.169068e-01,0.926397,35284.045028
75%,4.262783e-01,1.123887,35284.045028
max,4.310500e+00,2.500001,38230.115000
